<a href="https://colab.research.google.com/github/NonMagneticNeedle/reinforcement_learning/blob/master/muzero/Simple_MUZERO_Cartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**simple muzero without self play and reanalyze(target value network) and even not using proper montecarlo tree seach**

In [ ]:
#define models in pytorch
#define training loop and create data and augment with terminal states as absorbing states
#on policy training on single game trajectory
#define simple MTCS
#test model and simple MTCS


In [ ]:
import numpy as np
def stcat(x,support=300):
  x = np.sign(x) * ((abs(x) + 1)**0.5 - 1) + 0.001 * x
  x = np.clip(x, -support, support)
  floor = np.floor(x)
  prob = x - floor
  logits = np.zeros( 2 * support + 1)
  first_index = int(floor + support)
  second_index = int(floor + support+1)
  logits[first_index] = 1-prob
  if prob>0:
    logits[second_index] = prob
  return logits
def catts(x,support=300):
  support = np.arange(-support, support+1, 1)
  x = np.sum(support*x)
  x = np.sign(x) * ((((1 + 4 * 0.001 * (abs(x) + 1 + 0.001))**0.5 - 1) / (2 * 0.001))** 2- 1)
  return x  




In [ ]:
#doing training on single trajectory only

class Memory:
	def __init__(self):
		self.batch_s = []
		self.batch_a = []
		self.batch_r = []
		self.batch_s_ = []
		self.batch_done = []
		self.pred = []
		self.v = []

	def store(self, s, a, s_, r, done,pred,val):
		self.batch_s.append(s)
		self.batch_a.append(a)
		self.batch_r.append(r)
		self.batch_s_.append(s_)
		self.batch_done.append(done)
		self.pred.append(pred)
		self.v.append(val)  
	def clear(self):
		self.batch_s.clear()
		self.batch_a.clear()
		self.batch_r.clear()
		self.batch_s_.clear()
		self.batch_done.clear()
		self.pred.clear()
		self.v.clear()  
	def cnt_samples(self):
		return len(self.batch_s)
storage = Memory()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class representation(nn.Module):

    def __init__(self,observation_size,hidden_dim):
        super().__init__()
        self.observation_size=observation_size
        self.hidden_dim=hidden_dim
        #observation as input
        #representation function params
        self.h1 = nn.Linear(self.observation_size, 120)  
        self.h2 = nn.Linear(120,self.hidden_dim)
        self.h3 = nn.Linear(self.hidden_dim,self.hidden_dim)
                
    def forward(self, x):
        #x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.h1(x))
        x = F.relu(self.h2(x))
        x = F.sigmoid(self.h3(x))
        return x

class dynamics(nn.Module):

    def __init__(self,hidden_dim,action_dim,support=300):
        super().__init__()
        self.hidden_dim=hidden_dim
        self.action_dim=action_dim
        self.support=support        
        #hidden as input
        #dynamics        
        self.g1 = nn.Linear(self.hidden_dim , 120)#input current hidden state + action onehot concat
        self.g11 = nn.Linear(self.action_dim, 120)#input current hidden state + action onehot concat  
        self.gg = nn.Linear(2*120,2*120)
        self.g2 = nn.Linear(2*120,self.hidden_dim)
        self.g3 = nn.Linear(self.hidden_dim,self.hidden_dim)#next hidden state head
        self.g4 = nn.Linear(self.hidden_dim,2*self.support+1)#next reward head

        

    def forward(self, x,a):
        #x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.g1(x))
        a = F.relu(self.g11(a))
        x = torch.cat([x,a], dim=1)
        x = F.relu(self.gg(x))
        x = F.relu(self.g2(x))
        r = F.softmax(self.g4(x))#reward head
        x = F.sigmoid(self.g3(x))#next state head
        return x,r

class prediction(nn.Module):

    def __init__(self,hidden_dim,action_dim,support=300):
        super().__init__()
        self.hidden_dim=hidden_dim
        self.action_dim=action_dim
        self.support=support        
        #hidden as input  
        #prediction      
        self.p1 = nn.Linear(self.hidden_dim, 120)#input current hidden state + action onehot  
        self.p2 = nn.Linear(120,self.hidden_dim)
        self.p3 = nn.Linear(self.hidden_dim,self.action_dim)#policy head
        self.p4 = nn.Linear(self.hidden_dim,2*self.support+1)#value head

        

    def forward(self, x):
        #x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.p1(x))
        x = F.relu(self.p2(x))
        v = F.softmax(self.p4(x))
        p = F.softmax(self.p3(x))
        return p,v

class Net(nn.Module):

    def __init__(self,observation_size,hidden_dim,unroll_steps,action_dim):
        super().__init__()
        self.h=representation(observation_size,hidden_dim)
        self.g=dynamics(hidden_dim,action_dim)
        self.p=prediction(hidden_dim,action_dim)

    def initial_state(self, x):
        hout = self.h(x)
        p=v=0
        return hout#,p,v
    def next_state(self,hin,a):
        hout,r = self.g(hin,a)
        p=v=0
        return hout,r#,p,v
    def inference_initial_state(self, x):
        with torch.no_grad():
          hout = self.h(x)
          p=v=0
          return hout#,p,v
    def inference_next_state(self,hin,a):
        with torch.no_grad():
          hout,r = self.g(hin,a)
          p=v=0
          return hout,r#,p,v    

net = Net(observation_size=4,hidden_dim=126,unroll_steps=5,action_dim=2)
        

In [ ]:
def onehot(a,n):
  return np.eye(n)[a]


def returns(r,gamma = 0.9):
  #last reward is termial
  for i in range(len(r)):
    f = r[i:]
    sum = 0
    for j in range(len(f)):
      sum+=(gamma**j)*f[j]

    r[i]=stcat(sum)
  return r  

def q_returns(r,v_,d,gamma =0.8):
  vals=[]
  for i in range(len(r)):
    if d[i]:
      q = r[i] 
    else:
      q = r[i] + gamma*v_[i+1]
    vals.append(stcat(q))    

  return vals


def generate_training_data(storage,action_space,unroll_steps,no_of_absorbing_trials=50):
  #make the n step value return  
  #make absorbing states
  #make batches for training dynamics
  s=storage.batch_s
  ns=storage.batch_s_
  a=storage.batch_a
  r=storage.batch_r
  v = storage.v
  d=storage.batch_done
  r = q_returns(r,v,d)########################################### coverted rewards to returns
  p=storage.pred
  
  #v = get_return(r,d,ns)
  if d[-1]:
    terminal_state = ns[-1]
    terminal_reward = r[-1]#this will be value rather than reward and reward will be 0 after terminal state
    #make absorbing states using random actions
    for i in range(no_of_absorbing_trials):
      s.append(terminal_state)
      r.append(terminal_reward)#it will be value of terminal state that gets repeated rather than rewards
      a.append(np.random.randint(action_space))#append random actions
  for i in range(len(a)):
    a[i]= onehot(a[i],action_space)

  data_s = []
  data_r = []
  data_a= []
  for i in range(unroll_steps):
    data_s.append(s[i:i+(len(s)-unroll_steps)])
    data_r.append(r[i:i+(len(s)-unroll_steps)])
    data_a.append(a[i:i+(len(s)-unroll_steps)])
  storage.clear()  
  return data_s,data_r,data_a

optimizer = optim.Adam(net.parameters(), lr=0.001)
def train(net,s,r,a,epochs=10,batchsize=32,verbose=False):
    #only training dynamics on random games without scaling the dynamics losses by 1/unroll and stuff, need to watch video again

    initial_states=s[0]
    data_len = len(initial_states)
    unroll = len(s)
    gamma = 0
    for epoch in range(epochs):
      loss = 0
      for batch_indx in range(0,data_len,batchsize):
        optimizer.zero_grad()##################################check this is for model or for optimiser
        in_s =torch.tensor(initial_states[batch_indx:batch_indx+batchsize]).float()#device shit
        h = net.initial_state(in_s)
        batch_loss =torch.tensor(0)#to device i guess
        for u in range(unroll):
          #need to onehot actions
          act = torch.tensor(a[u][batch_indx:batch_indx+batchsize]).float()#to device
          h,pred_reward= net.next_state(h,act)
          true_reward= torch.tensor(r[u][batch_indx:batch_indx+batchsize]).float()#to device
          batch_loss =batch_loss +torch.mean(-((1-pred_reward)**gamma)*(true_reward*torch.log(pred_reward))) ##+ torch.mean((true_reward-pred_reward)**2) #+torch.mean(-(true_reward*torch.log(pred_reward)))
        batch_loss = batch_loss/unroll #scaling back to single step
        #net.zero_grad()##################################check this is for model or for optimiser
        loss+=batch_loss.item()
        #backwards
        batch_loss.backward()
        #optimiser step
        optimizer.step()
      if verbose:  
        print("epoch:",epoch,"loss:",loss)



In [ ]:
class Node():
  def __init__(self,node=None,child=0,action=None,depth=0):
    self.node =node
    self.action = action #action by parent that created it
    self.depth = depth
    self.children=[None]*child
    self.parents=[]
    self.reward = 0
    self.sum_rewards=0
  def expected_rewards(self):
    self.sum_rewards=self.reward
    #for p in self.parents:
    #  self.sum_rewards+=p.reward
    return self.sum_rewards#/(len(self.parents)+1) 

def dynamics(net,state,action):
  #print(state,action) 
  next_state,reward = net.inference_next_state(state,torch.tensor([action]).float())
  reward = catts(reward.numpy().ravel())
  return next_state,reward


def simple_minmax(net,current_observation,num_actions=2,depth=5,debug=False):
  curr_depth = 0
  history={}
  root = Node(current_observation,num_actions,0)#expand all actions to simulate all possibilities
  curr = [root]
  history[depth] = curr
  for d in range(1,depth+1):
    temp=[]
    if debug:
      print("no of current nodes :",len(curr))
    for p in curr:
      if debug:
        print("curr ",p.node)
      #p is a parent
      for action in range(len(p.children)):
        state,r = dynamics(net,p.node,onehot(action,num_actions))#target is environment specific for dynamics function
        if debug:  
          print("child",state,r)
        child = Node(state,num_actions,action,d)
        child.reward = r
        child.parents = p.parents.copy()
        child.parents.append(p)
        p.children[action]=child
        temp.append(p.children[action])
    if debug:  
      print("number of children : " ,len(temp))
    curr =temp
    if debug:
      print("_"*100)
  if debug:
    print("after search___")
    print("final number of children : " ,len(curr))
  max_avg_return = -1* 10**5
  best_child = None
  for c in curr:
    if debug:
      print(" children ", str(c.node))
    expected_rewards = c.expected_rewards()
    if expected_rewards>max_avg_return:
      max_avg_return = expected_rewards
      best_child = c
  if debug:    
    print("best child" ,best_child.node,best_child.depth,best_child.action,best_child.reward,max_avg_return)    
  action_to_take = best_child.parents[1].action
  if debug:
    print("best path")
    for i in range(len(best_child.parents)):
      print(best_child.parents[i].node)
    print(best_child.node)
    print("parent" ,best_child.parents[1].node)
    print("chose action", action_to_take)
  return action_to_take  
    


In [ ]:
import gym
import numpy as np
storage.clear()  
envs = ['CartPole-v1']

env=gym.make(envs[0])
#env=env.unwrapped

s_dim =env.observation_space.shape[0]
print("s_dim: ",s_dim)
a_dim =env.action_space.n
print("a_dim: ",a_dim)
a_bound =1 #env.action_space.high[0]
print("a_bound: ",a_bound)

episodes = 400
steps = 500
render=0
stop_train = False
for e in range(episodes):
  print(e," episode ","="*100)
  state = env.reset() 
  done = False
  step = 0 
  while not done:
    step+=1
    if render:
      env.render()
    h = net.inference_initial_state(torch.tensor([state]).float())  
    action = simple_minmax(net,h,a_dim,3) #env.action_space.sample() ######################################## Take smart actions now
    action_prob = onehot(action,a_dim)
    h_,f_r = net.inference_next_state(h,torch.tensor([action_prob]).float())
    f_r = catts(f_r.numpy().ravel())
    #f_r is q value of current state action pair now,rather than the reward due to state action
    next_state, reward, done, info = env.step(action)
    done = False
    if abs(next_state[2])<1.5:
      #reward=1/(1+abs(next_state[0]))
      reward=2*abs(1.5-abs(next_state[2]))/1.5
    #if abs(next_state[2])>0.1:
    #  reward=0
    if abs(next_state[0])>2 or abs(next_state[2])>1.5:
      done = True
      reward=-10
      #print(next_state)
    #print(reward,f_r)
    storage.store(state,action,next_state,reward,done,action_prob,f_r)
    state = next_state
    if step>steps:
      break
  print("steps ",step)    
  if step>100:
    stop_train = True

  if stop_train:
    storage.clear()  
    print("stopped training")
  

  if not stop_train:
    s,r,a=generate_training_data(storage,2,5,50)#storage,action_space,unroll_steps,no_of_absorbing_trials=50
    train(net,s,r,a,epochs=1,batchsize=64,verbose=False)



s_dim:  4
a_dim:  2
a_bound:  1
0  episode  ====================================================================================================
steps  22


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


1  episode  ====================================================================================================
steps  22
2  episode  ====================================================================================================
steps  22
3  episode  ====================================================================================================
steps  22
4  episode  ====================================================================================================
steps  23
5  episode  ====================================================================================================
steps  22
6  episode  ====================================================================================================
steps  22
7  episode  ====================================================================================================
steps  22
8  episode  ====================================================================================================
steps  22
9  episode  ====

In [ ]:
b = [i for i in range(10)]
print(b)
c = b[::-1]
print(b,c)
lst = [1,3,5]


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


TypeError: ignored

In [ ]:
#check the dynamics accuracy over large time steps
renew_state = 5
episodes = 400
steps = 500
render=0
for e in range(episodes):
  renew_state = 50
  print(e," episode ","="*100)
  state = env.reset() 
  done = False
  step = -1
  while not done:
    step+=1
    if render:
      env.render()
    if step == 0:
      print("renew")
      h = net.inference_initial_state(torch.tensor([state]).float())    
    if step>0 and step% renew_state ==0:  
      renew_state = renew_state*10
      print("renew")
      h = net.inference_initial_state(torch.tensor([state]).float())  
    action = simple_minmax(net,h,a_dim,4) #env.action_space.sample() ######################################## Take smart actions now
    action_prob = onehot(action,a_dim)
    h,f_r = net.inference_next_state(h,torch.tensor([action_prob]).float())
    f_r = catts(f_r.numpy().ravel())
    next_state, reward, done, info = env.step(action)
    done = False
    if abs(next_state[2])<1.5:
      #reward=1/(1+abs(next_state[0]))
      reward=2*abs(1.5-abs(next_state[2]))/1.5
    #if abs(next_state[2])>0.1:
    #  reward=0
    if abs(next_state[0])>2 or abs(next_state[2])>1.5:
      done = True
      reward=-10
      #print(next_state)
    print(reward,f_r)
    #storage.store(state,action,next_state,stcat(reward),done,action_prob)
    state = next_state
    if step>steps:
      break




